# Esercizio 6 - Ghigliottina

In [33]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer 
from collections import Counter

## Creo il corpus

Il mio corpus sarà composto da:
- i titoli di tutte le pagine italiane di Wikipedia
- i proverbi italiani presi dalla relativa pagina di Wikiquote
- i proverbi toscani presi dalla relativa pagina di Wikiquote
- i modi di dire italiani presi dalla relativa pagina di Wikiquote

## Ripulisco i file di input

Tramite la libreria *nltk* ripulisco tutti i file di input e li salvo in una lista che sarà il mio corpus.  
Impiega circa 1 minuto perchè i file sono molto grossi, produce un corpus di quasi 3 milioni di righe.

In [75]:
# Tokenize, remove stopwords, lemmatize all the input files
tokenizer = RegexpTokenizer(r'\w+')
stop_words = stopwords.words('italian')
lemmatizer = WordNetLemmatizer()

processed_corpus = []

def preprocess_file(path):
    preprocessed = []
    with open(path, 'r') as f:
        for line in f:
            line = line.lower()
            line = line.replace('_', ' ')
            tokens = tokenizer.tokenize(line)
            filtered = [w for w in tokens if not w in stop_words]
            filtered = [w for w in filtered if len(w) > 2] # Remove single and double letters
            filtered = [w for w in filtered if not w.isnumeric()] # Remove numbers
            filtered = [w for w in filtered if not any(char.isdigit() for char in w)] # Remove words with numbers
            lemmas = [lemmatizer.lemmatize(w) for w in filtered]
            preprocessed.append(lemmas)
    return preprocessed

# I use this to preprocess proverbs files because I wanto to remove the [Fonte x] part
def preprocess_proverbi(path):
    preprocessed = []
    with open(path, 'r') as f:
        for line in f:
            line = line.split('[')[0]
            line = line.lower()
            line = line.replace('_', ' ')
            tokens = tokenizer.tokenize(line)
            filtered = [w for w in tokens if not w in stop_words]
            filtered = [w for w in filtered if len(w) > 2] # Remove single and double letters
            filtered = [w for w in filtered if not w.isnumeric()] # Remove numbers
            filtered = [w for w in filtered if not any(char.isdigit() for char in w)] # Remove words with numbers
            lemmas = [lemmatizer.lemmatize(w) for w in filtered]
            preprocessed.append(lemmas)
    return preprocessed

titles_path = '../data/corpus-es6/itwiki-20221201-all-titles-in-ns0.txt'
modididire_path = '../data/corpus-es6/modi-di-dire.txt'
proverbi_path = '../data/corpus-es6/proverbi.txt'
proverbi_toscani_path = '../data/corpus-es6/proverbi-toscani.txt'

processed_corpus += preprocess_file(titles_path)
processed_corpus += preprocess_file(modididire_path)
processed_corpus += preprocess_proverbi(proverbi_path)
processed_corpus += preprocess_proverbi(proverbi_toscani_path)

print(len(processed_corpus))

2811279


## Idea

- Date le 5 parole di partenza, per ognuna di esse cerco all'interno del corpus tutte le linee che contengono quella parola.  
- Salvo tutte queste linee in una nuova lista, da cui rimuovo le 5 parole di partenza.
- Salvo in una struttura *Counter* tutte le parole che compaiono nella nuova lista.
- La sesta parola sarà quella che compare più frequentemente nella nuova lista.

In [59]:
# Search for all input words
def search_all(words, lines):
    result = []
    for line in lines:
        for word in words:
            if word in line:
                # Remove the word from the line -- Otherwise most probably it will be the most frequent word
                line = [w for w in line if w != word]
                result.append(line)
    freq_words = Counter()
    for line in result:
        freq_words.update(line)
    return result, freq_words

## Risultati su qualche esempio

Ho trovato i seguenti esempi dal sito *https://www.goccediperle.it/hobby/passatempi/il-gioco-della-parola-misteriosa/*, dotati anche di risultato corretto.  
I risultati ottenuti purtroppo non sono quasi mai corretti, però si può notare che spesso la parola proposta ha un chiaro nesso con qualcuna delle 5 parole di partenza.  
Inoltre ho notato che ogni qual volta compare la parola *cuore*, il risultato è sempre *sacro*.  
Questo mi fa pensare che il corpus sia in un qualche modo sbilanciato, come se la parola *cuore* sia presente in molti modi di dire e proverbi e molto spesso collegata alla parola *sacro*.  
Inoltre in qualche risultato compaiono parole inglesi, come *the*, *open* e *theft*. Queste vengono sicuramente dalla parte di corpus relativa all'elenco dei titoli delle pagine di Wikipedia.  
Questo dovrebbe contenere solamente pagine italiane, ma forse qualche titolo è in inglese, forse qualche pagina di canzoni o film ad esempio.  
Pertanto sicuramente il primo miglioramento possibile sarebbe quello di trovare un corpus migliore, più pulito, più bilanciato e che fornisca più conoscenza comune.  
Un'altra cosa che potrebbe migliorare le prestazioni sarebbe di considerare *n* parole più frequenti e non solo la prima, e magari confrontare i contesti in cui appaiono le parole proposte.  
Infine potrebbe risultare utile incrociare in qualche modo la ricerca sul corpus con anche ricerca di sensi, frame semantici, concetti, esempi di utilizzo, etc.

In [76]:
w1 = ['nemmeno', 'neppure', 'brutto', 'prete', 'natura']
r1 = 'scherzo'
w2 = ['preghiera', 'subito', 'rosso', 'verso', 'buona']
r2 = 'sera'
w3 = ['orario', 'portiere', 'via', 'film', 'libera']
r3 = 'uscita'
w4 = ['strada', 'don', 'frutto', 'taglio', 'cappuccetto']
r4 = 'bosco'
w5 = ['concerto', 'governo', 'fuori', 'quotidiano', 'lavatrice']
r5 = 'programma'
w6 = ['scena', 'maestro', 'caldo', 'freddo', 'schiena']
r6 = 'brivido'
w7 = ['mente', 'peggio', 'verdura', 'moda', 'prossimo']
r7 = 'passato'
w8 = ['pari', 'olio', 'guardia', 'biancheria', 'valuta']
r8 = 'cambio'
w9 = ['doppia', 'cuore', 'dopo', 'iva', 'carte']
r9 = 'partita'
w10 = ['tonno', 'medicina', 'parto', 'bionda', 'acqua']
r10 = 'naturale'
w11 = ['cercare', 'acida', 'gioco', 'unica', 'problema']
r11 = 'soluzione'
w12 = ['rubare', 'colpo', 'gelosia', 'delitto', 'muta']
r12 = 'scena'
w13 = ['cuore', 'capodanno', 'testa', 'passato', 'parata']
r13 = 'tuffo'
w14 = ['scuola', 'computer', 'chimica', 'tasse', 'tombola']
r14 = 'cartella'
w15 = ['notturno', 'ora', 'monte', 'treno', 'radio']
r15 = 'locale'
w16 = ['doppio', 'carta', 'soldi', 'pasta', 'regalo']
r16 = 'pacco'
w17 = ['ingannare', 'ammazzare', 'vola', 'spazio', 'pioggia']
r17 = 'tempo'
w18 = ['tutto', 'nobile', 'cucina', 'auto', 'lettura']
r18 = 'gas'

words = [w1, w2, w3, w4, w5, w6, w7, w8, w9, w10, w11, w12, w13, w14, w15, w16, w17, w18]
results = [r1, r2, r3, r4, r5, r6, r7, r8, r9, r10, r11, r12, r13, r14, r15, r16, r17, r18]

for i in range(len(words)):
    print(f'Le cinque parole sono: {words[i]}')
    found = search_all(words[i], processed_corpus)[1]
    sixth = found.most_common(1)[0][0]
    print(f'Ho trovato: {sixth}')
    print(f'La risposta corretta è: {results[i]}')
    if sixth == results[i]:
        print('Risposta corretta!')
    print('-'*80)

Le cinque parole sono: ['nemmeno', 'neppure', 'brutto', 'prete', 'natura']
Ho trovato: morta
La risposta corretta è: scherzo
--------------------------------------------------------------------------------
Le cinque parole sono: ['preghiera', 'subito', 'rosso', 'verso', 'buona']
Ho trovato: riserva
La risposta corretta è: sera
--------------------------------------------------------------------------------
Le cinque parole sono: ['orario', 'portiere', 'via', 'film', 'libera']
Ho trovato: the
La risposta corretta è: uscita
--------------------------------------------------------------------------------
Le cinque parole sono: ['strada', 'don', 'frutto', 'taglio', 'cappuccetto']
Ho trovato: statale
La risposta corretta è: bosco
--------------------------------------------------------------------------------
Le cinque parole sono: ['concerto', 'governo', 'fuori', 'quotidiano', 'lavatrice']
Ho trovato: orchestra
La risposta corretta è: programma
---------------------------------------------